In [675]:
# ----------------------------------------------------------------------------
# [Python-3.12] basic import for system check
# ----------------------------------------------------------------------------
import os
import sys
import time
import requests
import multiprocessing
import multiprocess
from collections import defaultdict
from functools import cache
from pathlib import Path
from glob import glob
# ----------------------------------------------------------------------------
# [Data-Plotting]
# ----------------------------------------------------------------------------
try:
    import matplotlib
    MATPLOTLIB_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package matplotlib is not installed on your system, verify or install\n")
    MATPLOTLIB_AVAILABLE = False
try:
    import tqdm
    TQDM_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package tqdm is not installed on your system, verify or install\n")
    TQDM_AVAILABLE = False
try:
    import pandas
    PANDAS_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package pandas is not installed on your system, verify or install\n")
    PANDAS_AVAILABLE = False
try:
    import seaborn
    SEABORN_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package seaborn is not installed on your system, verify or install\n")
    SEABORN_AVAILABLE = False
# ----------------------------------------------------------------------------
# [Import-checks]
# ----------------------------------------------------------------------------
print("\n")
print("MATPLOTLIB_AVAILABLE --- installed --->: ", MATPLOTLIB_AVAILABLE)
print("TQDM_AVAILABLE       --- installed --->: ", TQDM_AVAILABLE)
print("PANDAS_AVAILABLE     --- installed --->: ", PANDAS_AVAILABLE)
print("SEABORN_AVAILABLE    --- installed --->: ", SEABORN_AVAILABLE)
print("\n")
# ----------------------------------------------------------------------------
# [Path]
# ----------------------------------------------------------------------------
current_path = str(Path(sys.path[0]) / "..")
if current_path not in sys.path: sys.path.append(current_path)

APP_ROOT          = os.path.join(os.getcwd(), '..','..','..','..')
PROJECTNAME       = ""
POOL_COMPONENTDIR = ""
SOFTWARE          = "N/A"
SQL_DIR           = 'SQLFiles_sql'
APP_DATA_PATH     = "N/A"

#DATA_PATH         = os.path.join(APP_ROOT, '.','src','PythonCodes','data','Lumi')
DATA_PATH         = os.path.join(APP_ROOT, '..','LatticeRuns','Clusters','Lumi','LatticeRuns')
DATAPROCINTERCOM  = ""
TBLECNTS_DIR      = ""
SQL_FULLPATH_DIR  = os.path.join(os.getcwd())
# ----------------------------------------------------------------------------
# [SystemPath-Appens]
# ----------------------------------------------------------------------------
sys.path.append(APP_ROOT)
sys.path.append(APP_DATA_PATH)
sys.path.append(DATA_PATH)
sys.path.append(os.path.join(APP_ROOT, '.'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes','utils'))

print("Current Path  --->: ", current_path)
print("APP_ROOT      --->: ", APP_ROOT)
print("DATA_PATH     --->: ", DATA_PATH)
print("APP_DATA_PATH --->: ", APP_DATA_PATH)
# ----------------------------------------------------------------------------
# [Application]
# ----------------------------------------------------------------------------
#Application imports
import src.PythonCodes.DataManage_common
import src.PythonCodes.utils.messageHandler
import src.PythonCodes.utils.Command_line
import src.PythonCodes.DataManage_header



MATPLOTLIB_AVAILABLE --- installed --->:  True
TQDM_AVAILABLE       --- installed --->:  True
PANDAS_AVAILABLE     --- installed --->:  True
SEABORN_AVAILABLE    --- installed --->:  True


Current Path  --->:  C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..
APP_ROOT      --->:  C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..
DATA_PATH     --->:  C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..\..\LatticeRuns\Clusters\Lumi\LatticeRuns
APP_DATA_PATH --->:  N/A


In [676]:
version = src.PythonCodes.DataManage_common.DataManage_version()
c = src.PythonCodes.DataManage_common.DataManage_common()
rc = c.get_RC_SUCCESS()
# Getting the log file
logfile = c.getLogfileName()  #getting the name of the global log file
m = src.PythonCodes.utils.messageHandler.messageHandler(logfile = logfile)
# printing the header of Application
src.PythonCodes.DataManage_header.print_Bench_Grid_header(common=c, messageHandler=m)


 == [ ************************************************************************************] ==
 == [ *                    python code to run the Benchmarking suite.                    *] ==
 == [ *                                                                                  *] ==
 == [ * XXXXXX                          XX                XXXX             X        XX   *] ==
 == [ * X    X                          X               X    X                      X    *] ==
 == [ * X    X                          X              X                            X    *] ==
 == [ * X    X  XXXXX  XX XX    XXXXX   X XX           X       XXX XX   XXX     XXXXX    *] ==
 == [ * XXXXX  X     X  XX  X  X     X  XX  X          X         XX  X    X    X    X    *] ==
 == [ * X    X XXXXXXX  X   X  X        X   X  XXXXXXX X   XXX   X        X    X    X    *] ==
 == [ * X    X X        X   X  X        X   X          X     X   X        X    X    X    *] ==
 == [ * X    X X     X  X   X  X     X  X   X    

In [677]:
c.setApp_root(APP_ROOT)
c.setData_path(DATA_PATH)
c.setProjectName(PROJECTNAME)
c.setPool_componentdir(POOL_COMPONENTDIR)
c.setSoftware(SOFTWARE)
c.setDataProcInterCom(DATAPROCINTERCOM)
c.setJSon_TableCounts_Dir(TBLECNTS_DIR)
c.setSql_dir(SQL_DIR)
c.setSql_fullPath_dir(SQL_FULLPATH_DIR)

In [678]:
# --------------------------------------------------------------------------
# [Main-code]
# --------------------------------------------------------------------------
m.printMesgStr("This is the main program      :", c.getCyan(), "Bench_Grid_HiRep.py")

 == [This is the main program      : Bench_Grid_HiRep.py] ==


In [679]:
target_file_default = c.getTarget_File()
m.printMesgStr("Default target file           :", c.getMagenta(), target_file_default)

machine_name = "Lumi"
batch_action = "BKeeper_run_gpu"
simulation_size="small"

msg_analysis = c.getTarget_File().split(".txt")[0] + c.undr_scr + \
                batch_action    + c.undr_scr                    + \
                simulation_size + c.undr_scr                    + \
                "batch_files"   + c.txt_ext
c.setTarget_File(str(msg_analysis))
m.printMesgStr("Target file for analysis      :", c.getMagenta(), c.getTarget_File())

c.setTargetdir( os.path.join(c.getData_path(), c.getTarget_File()))
m.printMesgStr("Full Path target file         :", c.getCyan(), c.getTargetdir())

if Path(c.getTargetdir()).is_file():
    m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), "Exists")

 == [Default target file           : target.txt] ==
 == [Target file for analysis      : target_BKeeper_run_gpu_small_batch_files.txt] ==
 == [Full Path target file         : C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..\..\LatticeRuns\Clusters\Lumi\LatticeRuns\target_BKeeper_run_gpu_small_batch_files.txt] ==
    [Check]: target file       --->: Exists


In [680]:
# --------------------------------------------------------------------------
# [Data-Screening]
# --------------------------------------------------------------------------
def getTarget_file_lst(c, m, target_file):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_lst = []
    target_file_dir = []
    with open(target_file) as file:
        cnt = 0
        for line in file:
            target_file_lst.append(os.path.basename(line).strip())
            target_file_dir.append(os.path.dirname(os.path.realpath(line)).strip())
            cnt += 1
        # [end-For-Loop]
        m.printMesgAddStr("Number of files in target_file : ", c.getYellow(), str(cnt))
    # [end-with]
    return rc, target_file_lst, target_file_dir
# [end-function]
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst, target_file_dir = getTarget_file_lst(c, m, c.getTargetdir())
#print(target_file_lst)
#print(target_file_dir)


 == [Getting target file list      : getTarget_file_lst] ==
    Number of files in target_file : 51


In [681]:
def getTarget_file_lst(c, m, target_file_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_cluster_lst = []
    for i in range(len(target_file_lst[:])):
        msg = ( os.path.join(c.getData_path(), batch_action, simulation_size, str(target_file_lst[i].split(".sh")[0]), target_file_lst[i]) ).strip()
        if Path(msg).is_file():
            m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), c.getMagenta()+ msg + c.getGreen() + " ---> Exists")
            # Now get the output file to analise and put it into a list
            cluster_out_file = msg.split(".sh")[0]+".out"
            # Extracting cluster files that has been benched
            if Path(cluster_out_file).is_file():
                m.printMesgAddStr("[Check]: Cluster file      --->: ", c.getGreen(), c.getYellow()+ cluster_out_file + c.getGreen() + " ---> Exists")
                target_file_cluster_lst.append(cluster_out_file)
            # [end-if]
        # [end-if]
    # [end-for-loop]
    return rc, target_file_cluster_lst
# [end-function]
# --------------------------------------------------------------------------
rc, target_file_cluster_lst = getTarget_file_lst(c, m, target_file_lst[:])

 == [Getting target file list      : getTarget_file_lst] ==
    [Check]: target file       --->: C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..\..\LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi01-01-04-04_small.sh ---> Exists
    [Check]: target file       --->: C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..\..\LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-02-02-01_small.sh ---> Exists
    [Check]: target file       --->: C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..

In [682]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr("Data extraction cluster out   :", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst[:] --->: ", c.getYellow(), target_file_cluster_lst[:])

def readCluster_file_out(c, m, target_file_cluster_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    end_key = "###############################################"
    bench_dict = {}
    start_key_lst = [
        'Performing benchmark for SU(2), adjoint',
        'Perf0orming benchmark for SU(2), fundamental',
        'Performing benchmark for SU(3), fundamental',
        'Performing benchmark for Sp(4), fundamental'
    ]
    # Starting the parsing of files over the start_key_lst
    for i in range(len(target_file_cluster_lst[:])):
        #with open(target_file_cluster_lst[i]) as cluster_file:
        cluster_file = open(target_file_cluster_lst[i])
        lines = cluster_file.readlines()
        database_file_len = len(lines)
        print("the number of lines in file "+ str(target_file_cluster_lst[i]) + " is : "+str(database_file_len))
        start_key_su2_adjoint = "Performing benchmark for SU(2), adjoint"
        appending = False
        cnt = 0
        insert_msg = ""
        for j in range(database_file_len):
            if start_key_su2_adjoint in lines[j].split('\n')[0]:
                appending = True
            if end_key in lines[j].split('\n')[0]:
                appending = False
            elif appending:
                print(lines[j].split('\n')[0])


            # [end-if]
        # [end-for-loop [j]]
    # [end-for-loop [i]]
    return rc
# [end-function]
# --------------------------------------------------------------------------
rc = readCluster_file_out(c, m, target_file_cluster_lst)


# Continue from here with the delimiter method.



 == [Data extraction cluster out   : Lumi] ==
    Simulation size                : small
    target_file_cluster_lst[:] --->: ['C:\\cygwin64\\home\\Frederic\\SwanSea\\SourceCodes\\Bench_Grid_HiRep\\src\\PythonCodes\\Jupyter-NoteBooks\\LLR-Bench_Grid_HiRep_Notebooks\\..\\..\\..\\..\\..\\LatticeRuns\\Clusters\\Lumi\\LatticeRuns\\BKeeper_run_gpu\\small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-01-02-02_small\\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-01-02-02_small.out']
 == [Getting target file list      : readCluster_file_out] ==
the number of lines in file C:\cygwin64\home\Frederic\SwanSea\SourceCodes\Bench_Grid_HiRep\src\PythonCodes\Jupyter-NoteBooks\LLR-Bench_Grid_HiRep_Notebooks\..\..\..\..\..\LatticeRuns\Clusters\Lumi\LatticeRuns\BKeeper_run_gpu\small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-01-02-02_small\Run_BKeeper_run_gpu_lat24.24.24.32_nodes002_mpi04-01-02-02_small.out is : 198
BKeeper : Message  : 0.139000 s : # Performing benchmark for SU(2), adjoin